#**Advanced model training using hyperopt**

In the Advanced Model Training tutorial we have already taken a look into hyperparameter optimasation using GridHyperparamOpt in the deepchem pacakge. In this tutorial, we will take a look into another hyperparameter tuning library called hyperopt.

## Colab

This tutorial and the rest in this sequence can be done in Google colab. If you'd like to open this notebook in colab, you can use the following link.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/deepchem/deepchem/blob/master/examples/tutorials/Advanced_model_training_using_hyperopt.ipynb)

## Setup

To run DeepChem and Hyperopt within Colab, you'll need to run the following installation commands. You can of course run this tutorial locally if you prefer. In that case, don't run these cells since they will download and install DeepChem and Hyperopt in your local machine again.




In [1]:
!pip install deepchem
!pip install hyperopt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 693.2/693.2 KB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 MB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 29.4/29.4 MB 22.9 MB/s eta 0:00:00
  Attempting uninstall: scipy
    Found existing installation: scipy 1.10.1
    Uninstalling scipy-1.10.1:
      Successfully uninstalled scipy-1.10.1
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Hyperparameter Optimization via hyperopt

Let's start by loading the HIV dataset.  It classifies over 40,000 molecules based on whether they inhibit HIV replication.



In [2]:
import deepchem as dc
tasks, datasets, transformers = dc.molnet.load_hiv(featurizer='ECFP', splitter='scaffold')
train_dataset, valid_dataset, test_dataset = datasets


[07:22:09] WARNING: not removing hydrogen atom without neighbors
[07:22:09] WARNING: not removing hydrogen atom without neighbors
[07:22:40] WARNING: not removing hydrogen atom without neighbors
[07:22:40] WARNING: not removing hydrogen atom without neighbors


Now, lets import the hyperopt library, which we will be using to fund the best parameters

In [3]:
from hyperopt import hp, fmin, tpe, Trials

Then we have to declare a dictionary with all the hyperparameters and their range that you will be tuning them in. This dictionary will serve as the search space for the hyperopt. 
Some basic ways of declaring the ranges in the dictionary are:



*   hp.choice('label',[*choices*]) : this is used to specify a list of choices
*   hp.uniform('label' ,low=*low_value* ,high=*high_value*) :  this is used to specify a uniform distibution between the low and high values. The values between them can be any real number, not necessaarily an integer.

Here, we are going to use a multitaskclassifier to classify the HIV dataset and hence the appropriate search space is as follows.






In [4]:
search_space = {
    'layer_sizes': hp.choice('layer_sizes',[[500], [1000], [2000], [1000,1000]]),
    'dropouts': hp.uniform('dropout', low=0.2, high=0.5),
    'learning_rate': hp.uniform('learning_rate', high=0.001, low=0.0001)
}

We should then declare a function to be minimized by the hyperopt. So, here we should use the function to minimize our multitaskclassifier model. Additionally, we are using a validation callback to validate the classifier for every 1000 steps, then we are passing the best score as the return. The metric used here is 'roc_auc_score', which needs to be maximized. To maximize a non-negative value is equivalent to minimize its opposite number, hence we are returning the negative of the validation score.




In [5]:
import tempfile
#tempfile is used to save the best checkpoint later in the program.

metric = dc.metrics.Metric(dc.metrics.roc_auc_score)

def fm(args):
  save_dir = tempfile.mkdtemp()
  model = dc.models.MultitaskClassifier(n_tasks=len(tasks),
                      n_features=1024,
                      layer_sizes=args['layer_sizes'],
                      dropouts=args['dropouts'],
                      learning_rate=args['learning_rate'])
  #validation callback that saves the best checkpoint, i.e the one with the maximum score.
  validation=dc.models.ValidationCallback(valid_dataset, 1000, [metric], save_dir=save_dir, transformers=transformers, save_on_minimum=False)
  
  model.fit(train_dataset, nb_epoch=25, callbacks=validation)

  #restoring the best checkpoint and passing the negative of its validation score to be minimized.
  model.restore(model_dir=save_dir)
  valid_score = model.evaluate(valid_dataset, [metric], transformers)

  return -1 * valid_score['roc_auc_score']

Here, we are calling the fmin function of the hyperopt, where we pass on the function to be minimized, the algorithm to be followed, max number of evals and a trials object. The Trials object is used to keep All hyperparameters, loss, and other information, this means you can access them after running optimization. Also, trials can help you to save important information and later load and then resume the optimization process.

Moreover, for the algorithm there are three choice which can be used without any additional configuration. they are :-  


*   Random Search - rand.suggest
*   TPE (Tree Parzen Estimators) - tpe.suggest
*   Adaptive TPE - atpe.suggest






In [6]:
trials=Trials()
best = fmin(fm,
    		space=search_space,
    		algo=tpe.suggest,
    		max_evals=15,
    		trials=trials)


  0%|          | 0/15 [00:00<?, ?it/s, best loss: ?]Step 1000 validation: roc_auc_score=0.738666
Step 2000 validation: roc_auc_score=0.759833
Step 3000 validation: roc_auc_score=0.750727
Step 4000 validation: roc_auc_score=0.767338
Step 5000 validation: roc_auc_score=0.750678
Step 6000 validation: roc_auc_score=0.751681
Step 7000 validation: roc_auc_score=0.727666
Step 8000 validation: roc_auc_score=0.73623
  7%|▋         | 1/15 [06:15<1:27:38, 375.61s/it, best loss: -0.7673381466784244]Step 1000 validation: roc_auc_score=0.731238
Step 2000 validation: roc_auc_score=0.759585
Step 3000 validation: roc_auc_score=0.756392
Step 4000 validation: roc_auc_score=0.760106
Step 5000 validation: roc_auc_score=0.764393
Step 6000 validation: roc_auc_score=0.762134
Step 7000 validation: roc_auc_score=0.765014
Step 8000 validation: roc_auc_score=0.763192
 13%|█▎        | 2/15 [11:49<1:16:03, 351.04s/it, best loss: -0.7673381466784244]Step 1000 validation: roc_auc_score=0.736817
Step 2000 validation: 

The code below is used to print the best hyperparameters found by the hyperopt.

In [7]:
print("Best: {}".format(best))


Best: {'dropout': 0.24165404739692975, 'layer_sizes': 1, 'learning_rate': 0.00023508803192311167}


The hyperparameter found here may not be necessarily the best one, but gives a general idea on which parameters are effective. To get mroe accurate results, one has to increase the number of validation epochs and the epochs the model fit. But doing so may increase the time in finding the best hyperparameters.

# Congratulations! Time to join the Community!

Congratulations on completing this tutorial notebook! If you enjoyed working through the tutorial, and want to continue working with DeepChem, we encourage you to finish the rest of the tutorials in this series. You can also help the DeepChem community in the following ways:

## Star DeepChem on [GitHub](https://github.com/deepchem/deepchem)
This helps build awareness of the DeepChem project and the tools for open source drug discovery that we're trying to build.

## Join the DeepChem Gitter
The DeepChem [Gitter](https://gitter.im/deepchem/Lobby) hosts a number of scientists, developers, and enthusiasts interested in deep learning for the life sciences. Join the conversation!